In [48]:
from bigml.api import BigML
from pandas import DataFrame, to_numeric
import numpy
from pprint import pprint
from math import isnan
import dataprocessing

In [49]:
big_ml_project = 'project/5d94a428eba31d460c00023f'
api = BigML(project=big_ml_project)

In [50]:
model_id = 'ensemble/5db3a3a55299631c8200a5ed'

In [51]:
input = {
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}

In [52]:
if 'MonthlyIncome' not in input:
    input['MonthlyIncome'] = None

Apply all feature mods.

In [53]:
from tools import json_save_project, json_load_project
project = 'gmsc'
version = 'v1.3'
project_data = json_load_project(project, version)
pprint(project_data)

{'fulltrain_csv': 'cs-training.csv',
 'mods': ['monthly_income_fillna'],
 'processing': [{'arguments': {'column': 'MonthlyIncome', 'value': 0},
                 'function': 'process_fillna'},
                {'arguments': {'column': 'DebtRatio', 'value': 0.0},
                 'function': 'process_fillna'},
                {'arguments': {'debt_ratio_fill_value': None},
                 'function': 'monthly_debt'},
                {'arguments': {}, 'function': 'balanced_income'},
                {'arguments': {'column': 'NumberOfDependents', 'value': 0},
                 'function': 'process_fillna'},
                {'arguments': {}, 'function': 'income_per_hm'},
                {'arguments': {}, 'function': 'late_category'},
                {'arguments': {'weight': [1, 2, 3]}, 'function': 'late_score'},
                {'arguments': {'columns': ['NumberOfTime30-59DaysPastDueNotWorse',
                                           'NumberOfTime60-89DaysPastDueNotWorse',
                  

In [54]:
# import import_ipynb
# import features

In [55]:
df  = DataFrame(input, index=[0])

In [56]:
for process in project_data['processing']:
    getattr(dataprocessing, process['function'])(df, **process['arguments'])

In [57]:
input2 = df.iloc[0].to_dict()
pprint(input)

{'DebtRatio': 0,
 'MonthlyIncome': None,
 'NumberOfDependents': 1,
 'NumberOfOpenCreditLinesAndLoans': 6,
 'NumberOfTime30-59DaysPastDueNotWorse': 1,
 'NumberOfTime60-89DaysPastDueNotWorse': 0,
 'NumberOfTimes90DaysLate': 0,
 'NumberRealEstateLoansOrLines': 1,
 'RevolvingUtilizationOfUnsecuredLines': 0.01703559,
 'age': 42}


In [58]:
for key in input2:
    if isinstance(input2[key], numpy.int64):
        input2[key] = int(input2[key])
    if isinstance(input2[key], numpy.float64):
        input2[key] = float(input2[key])
    if isinstance(input2[key], int) or isinstance(input2[key], float):
        if isnan(input2[key]):
            input2[key]= 'NA'
#    print(type(input2[key]))
pprint(input2)

{'BalancedIncome': 'NA',
 'DebtRatio': 'NA',
 'IncomePerHouseholdMember': 'NA',
 'LateCategory': 'L0',
 'LateScore': 1.0,
 'MonthlyDebt': 0,
 'MonthlyIncome': 0,
 'NumberOfOpenCreditLinesAndLoans': 6,
 'NumberRealEstateLoansOrLines': 1,
 'RevolvingUtilizationOfUnsecuredLines': 0.01703559,
 'age': 42}


In [59]:
model = api.get_ensemble(model_id)

In [60]:
prediction = api.create_prediction(model, input2)
#prediction

In [61]:
print(f"prediction: {prediction['object']['output']}")
print(f"confidence: {prediction['object']['confidence']}")

prediction: 0
confidence: 0.9477
